In [ ]:
###############################################################################
# This notebook provides some tools for better integration between the        #
# Pacific EMIS and Pacific SIS. It does the following                         #
#   - Sync look values from Pacific EMIS to Pacific SIS deployments           #
###############################################################################

# Core stuff
import os
import json
import datetime as dt

# Data stuff
import pandas as pd # Data analysis
import xlrd # excel 
import pyodbc # SQL DB

# Pretty printing stuff
from IPython.display import display, HTML
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Initial setup
cwd = os.getcwd()

# Configuration
with open('config.json', 'r') as file:
     config = json.load(file)
        
# SIS config
sis_database = config['sis_database']
sis_tenant_id = config['sis_tenant_id']
sis_user_guid = config['sis_user_guid']
sis_field_name = config['sis_field_name']
sis_lov_name = config['sis_lov_name']

# Config
country = config['country']
datetime = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# MS SQL Server connection
conn = """
    Driver={{ODBC Driver 17 for SQL Server}};
    Server={},{};
    Database={};
    authentication=SqlPassword;UID={};PWD={};
    TrustServerCertificate=yes;
    autocommit=True
    """.format(config['emis_server_ip'], config['emis_server_port'], config['emis_database'], config['emis_uid'], config['emis_pwd'])

ms_sql_conn = pyodbc.connect(conn)

# MySQL Connection
import mysql.connector
from mysql.connector import errorcode

mysql_conn = mysql.connector.connect(user=config['sis_user'],                              
                                password=config['sis_pwd'],
                                database=config['sis_database'],
                                host=config['sis_host'],
                                port=config['sis_server_port'])

In [ ]:
# Load the relevant data from EMIS database
query = """
SELECT [schNo] AS SCHOOLID
      ,[schName] AS SCHOOLNAME
	  ,I.iName AS ISLAND
	  ,D.dName AS DISTRICT
	  ,A.authName AS AUTHORITY
	  ,AT.codeDescription AS AUTHORITYTYPE
	  ,AG.codeDescription AS URBAN
      ,[schClosed] AS CLOSED
      ,[schCloseReason] AS CLOSEDREASON
  FROM [dbo].[Schools] S
  INNER JOIN Islands I ON S.iCode = I.iCode
  INNER JOIN Districts D ON I.iGroup = D.dID
  INNER JOIN Authorities A ON S.schAuth = A.authCode
  INNER JOIN lkpAuthorityType AT ON A.authType = AT.codeCode
  INNER JOIN lkpAuthorityGovt AG ON AT.codeGroup = AG.codeCode
"""

query_ethnicity = """SELECT [codeCode], [codeDescription], [codeGroup], [codeSeq] FROM [dbo].[lkpEthnicity]"""
query_ethnicity_group = """SELECT [codeCode], [codeDescription], [codeSeq] FROM [dbo].[lkpEthnicityGroup]"""
                          
df_schools_x = pd.read_sql(query, ms_sql_conn)
display(df_schools_x.head(3))
df_ethnicities = pd.read_sql(query_ethnicity, ms_sql_conn)
#display(df_ethnicities.head(3))
df_ethnicities_groups = pd.read_sql(query_ethnicity_group, ms_sql_conn)
#display(df_ethnicities_groups.head(3))
df_lookup_values = {
    'ethnicity': df_ethnicities,
    'race': df_ethnicities_groups
}

In [ ]:
# Load the core data from SIS database that is always to be loaded
query_schools = """
SELECT * FROM school_master;
"""

query_dpdown_valuelist_next_id = """
SELECT max(id) FROM opensisv2_ef6.dpdown_valuelist;
"""

query_fields_category = """
SELECT * FROM opensisv2_ef6.fields_category;
"""

dpdown_valuelist_next_id = None
cursor = mysql_conn.cursor()
cursor.execute(query_dpdown_valuelist_next_id)
for r in cursor:
    print('last dpdown_valielist ID: ', r[0])
    dpdown_valuelist_next_id = r[0]
cursor.close()

df_schools_sis = pd.read_sql(query_schools, mysql_conn)
display(df_schools_sis.head(3))

df_fields_category_sis = pd.read_sql(query_fields_category, mysql_conn)
display(df_fields_category_sis.head(3))

In [ ]:
# Load specific lookup data of interest (just for quick viewing, not needed actually)
query_custom_fields = """
SELECT * FROM opensisv2_ef6.custom_fields WHERE field_name = '{}';
""".format(sis_field_name)

query_dpdown_valuelist = """
SELECT * FROM opensisv2_ef6.dpdown_valuelist WHERE lov_name = '{}';
""".format(sis_lov_name)

df_custom_fields = pd.read_sql(query_custom_fields, mysql_conn)
display(df_custom_fields.head(3))

df_dpdown_valuelist = pd.read_sql(query_dpdown_valuelist, mysql_conn)
display(df_dpdown_valuelist.head(3))

In [ ]:
# Make a simple report to see what will change (what was already in there vs what will be synced)
# Mostly for visual inspection and analysis of possible repercussions.
print("EMIS Lookup")
display(df_lookup_values[sis_field_name])

print("SIS existing values")
df_dpdown_valuelist_first_school = df_dpdown_valuelist[df_dpdown_valuelist['school_id'] == 1]
df_dpdown_valuelist_first_school = df_dpdown_valuelist_first_school[['lov_code', 'lov_column_value', 'sort_order']]
df_dpdown_valuelist_first_school.rename(columns = {'lov_code':'codeCode', 'lov_column_value':'codeDescription', 'sort_order':'codeSeq'}, inplace = True)
display(df_dpdown_valuelist_first_school)

emis_values = set(df_lookup_values[sis_field_name]['codeDescription'].values)
sis_values = set(df_dpdown_valuelist_first_school['codeDescription'].values)
emis_values - sis_values

In [ ]:
# TODO - Get all associations for lookups values to be removed and generate the necessary update statements to use the new lookups

In [ ]:
# Holds dataframes for any possible EMIS lookup values to be synced

# IMPORTANT - Currently the supported format is an array of tuples with the data (description, sort_order)
# probably should be (code, description, sort_order)

# The EMIS ethnicities to sync to the SIS
df = df_ethnicities[['codeCode', 'codeDescription', 'codeSeq']]
ethnicities = list(df.itertuples(index=False, name=None))
df = df_ethnicities_groups[['codeCode', 'codeDescription', 'codeSeq']]
ethnic_groups = list(df.itertuples(index=False, name=None))
lookup_values = {
    'ethnicity': ethnicities,
    'race': ethnic_groups
}

In [ ]:
# List of existing ids used in dpdown_valuelist. Let's collect for re-use
dpdown_valuelist_ids = list(df_dpdown_valuelist['id'].values)
# The SIS schools
school_ids = list(df_schools_sis['school_id'].values)
# Handling of primary key
next_id = dpdown_valuelist_next_id

filename = 'data/' + country + '/' + sis_field_name + '-sync-script.sql'
file = open(filename, "w") 

file.write("USE {};\n\n".format(sis_database))

# Remove all existing ethnicities in the SIS in a rolled back transaction
file.write("START TRANSACTION;\n\n")

# Some basic summary verification
file.write("SELECT school_id, count(lov_name) num_{} FROM dpdown_valuelist WHERE lov_name = '{}' GROUP BY school_id;\n\n".format(sis_field_name, sis_lov_name))

# The actual deletion
file.write("DELETE FROM dpdown_valuelist WHERE lov_name = '{}';\n\n".format(sis_lov_name))

# Re-inserts from the EMIS values
for school_id in school_ids:
    # insert statement for all the lookup_values to sync (e.g. ethnicities)
    for lookup in lookup_values[sis_field_name]:
        if len(dpdown_valuelist_ids) != 0:
            # still some ids for re-use
            id = dpdown_valuelist_ids.pop(0)
        else:
            # continue with new IDs
            next_id = next_id + 1
            id = next_id
        insert_str = "INSERT INTO dpdown_valuelist(id, tenant_id, school_id, lov_name, lov_column_value, lov_code, sort_order, created_by, created_on, updated_by, updated_on) VALUES ({}, '{}', {}, '{}', '{}', '{}', {}, '{}', '{}', '{}', '{}');\n".format(id, sis_tenant_id, school_id, sis_lov_name, lookup[1], lookup[0], lookup[2], sis_user_guid, datetime, sis_user_guid, datetime)    
        #print(insert_str)
        file.write(insert_str)
        
# Some basic summary verification
file.write("\nSELECT school_id, count(lov_name) num_{} FROM dpdown_valuelist WHERE lov_name = '{}' GROUP BY school_id;\n\n".format(sis_field_name, sis_lov_name))        

# Default to ROLLBACK. Final step is examination of the load script, test in development and then COMMIT when certain.
file.write("\nROLLBACK;")
        
file.close()

In [ ]:
# Close database connections
mysql_conn.close()